In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tokenizernofre/tokenizer.json
/kaggle/input/tokenizernofre/tokenizer_config.json
/kaggle/input/tokenizernofre/special_tokens_map.json
/kaggle/input/tokenizernofre/vocab.txt
/kaggle/input/tokenizerfre/tokenizer.json
/kaggle/input/tokenizerfre/tokenizer_config.json
/kaggle/input/tokenizerfre/special_tokens_map.json
/kaggle/input/tokenizerfre/vocab.txt
/kaggle/input/bert_nofre_model/pytorch/default/1/config.json
/kaggle/input/bert_nofre_model/pytorch/default/1/model.safetensors
/kaggle/input/bert_fre_model/pytorch/default/1/config.json
/kaggle/input/bert_fre_model/pytorch/default/1/model.safetensors


In [5]:
tokenizerfre_dir='/kaggle/input/tokenizerfre'
tokenizernofre_dir='/kaggle/input/tokenizernofre'

bert_model_fre_dir='/kaggle/input/bert_nofre_model/pytorch/default/1'
bert_model_nofre_dir='/kaggle/input/bert_nofre_model/pytorch/default/1/'

In [6]:
from transformers import BertTokenizer
# 从本地路径加载tokenizer
tokenizer = BertTokenizer.from_pretrained(tokenizerfre_dir)

# 使用方式同上
encoded_input = tokenizer("Your text here", return_tensors='pt')
encoded_input

{'input_ids': tensor([[  101,   100, 10539, 10815,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [7]:
import torch
from transformers import BertForSequenceClassification, BertConfig


# 加载模型 和配置
# 
config = BertConfig.from_pretrained(bert_model_fre_dir)
model = BertForSequenceClassification.from_pretrained(
    bert_model_fre_dir,
    config=config,
    use_safetensors=True
)

model.eval()  

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
def predict_with_model(model, texts, tokenizer, device, label_map=None):
    """
    使用训练好的模型进行预测
    
    参数:
    - model: 已训练的分类模型
    - texts: 待预测的文本列表
    - tokenizer: 分词器
    - device: 计算设备
    - label_map: 类别索引到名称的映射字典
    
    返回:
    - 包含预测类别、概率的列表
    """
    model.eval()
    results = []
    
    # 处理单条文本的情况
    if isinstance(texts, str):
        texts = [texts]
    
    with torch.no_grad():
        for text in texts:
            # 编码文本
            inputs = tokenizer(text, return_tensors="pt").to(device)
            
            # 模型预测
            outputs = model(**inputs)
            logits = outputs.logits
            
            # 计算概率分布
            probs = torch.nn.functional.softmax(logits, dim=1)
            confidence, predicted_class_id = torch.max(probs, dim=1)
            
            # 转换为numpy
            predicted_class_id = predicted_class_id.item()
            confidence = confidence.item()
            
            # 映射类别名称（如果提供了映射）
            predicted_label = label_map[predicted_class_id] if label_map else predicted_class_id
            
            results.append({
                'text': text,
                'predicted_class_id': predicted_class_id,
                'predicted_label': predicted_label,
                'confidence': confidence,
                'probabilities': probs.cpu().numpy()[0].tolist()
            })
    
    return results


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:

# 使用示例
label_map = {0: "负面", 1: "正面"}  # 根据实际情况修改
cmt_test=['商家发出的根本就不是一款，跟链接里的不一样，买的支架款也没有支架，真服了，发货巨慢，售后开个发票也磨磨叽叽到现在也没有拿到',     
          '保温效果一般，两个小时不到就凉了',
          '包装有脏污，盒子封口贴全是灰尘，送人用的，没办法只能把封口贴撕下来'
          ,'包裹运输中破损，持续20多天都没有退款',
          '纯纯塑料杯子，这杂能用，一股塑料的味道，劣质塑料',
          '有味道，不适合给孩子用，也不适合放热水，就是好看而已，真的一分钱一分货',
          '收到 货 之后 光顾 查询 序列号 去 没 仔细检查 外观 激活 之后 贴膜 才 发现 背 logo 上面 一点 刮伤 很 郁闷 很想 知道 刮痕',
          '风扇 噪音 大，没 想象 的 那么好',
          '这个保温杯真的超实用，670ml大容量满足全天饮水需求，316不锈钢材质安全无异味，保温效果长达12小时以上，粉色外观清新时尚。',
          '设计很漂亮，尺寸大小合适，包装仔细完整，手感很不错',
          '安装简单，价格实惠，宝贝确实不错，风力蛮大',
          '颜值高，白色很耐看，最喜欢的是米家的app远程控制，虽然需要有小爱音箱才能连接米家app,但是太好用了'
          ,'风量大声音小，可以调节风量，风速比较均匀，吹起来很舒服很自然',
         '期盼多久的镜子，太失望了',
          '包装有问题，把我手划破了',
          '除了好看一无是处',
          '太小了，装不了什么',]

model.to(device)
predictions = predict_with_model(model, cmt_test , tokenizer, device, label_map)

# 打印详细预测结果
for pred in predictions:
    print(f"文本: {pred['text']}")
    print(f"预测类别: {pred['predicted_label']} (索引:{pred['predicted_class_id']})")
    print(f"置信度: {pred['confidence']:.4f}")
    print(f"概率分布: {pred['probabilities']}")
    print("-" * 50)

文本: 商家发出的根本就不是一款，跟链接里的不一样，买的支架款也没有支架，真服了，发货巨慢，售后开个发票也磨磨叽叽到现在也没有拿到
预测类别: 负面 (索引:0)
置信度: 0.9994
概率分布: [0.9994184970855713, 0.0005814721807837486]
--------------------------------------------------
文本: 保温效果一般，两个小时不到就凉了
预测类别: 负面 (索引:0)
置信度: 0.9985
概率分布: [0.998454213142395, 0.0015457230620086193]
--------------------------------------------------
文本: 包装有脏污，盒子封口贴全是灰尘，送人用的，没办法只能把封口贴撕下来
预测类别: 负面 (索引:0)
置信度: 0.9948
概率分布: [0.994773805141449, 0.005226248409599066]
--------------------------------------------------
文本: 包裹运输中破损，持续20多天都没有退款
预测类别: 负面 (索引:0)
置信度: 0.9962
概率分布: [0.996221661567688, 0.0037782927975058556]
--------------------------------------------------
文本: 纯纯塑料杯子，这杂能用，一股塑料的味道，劣质塑料
预测类别: 负面 (索引:0)
置信度: 0.9987
概率分布: [0.9986698627471924, 0.0013301356229931116]
--------------------------------------------------
文本: 有味道，不适合给孩子用，也不适合放热水，就是好看而已，真的一分钱一分货
预测类别: 正面 (索引:1)
置信度: 0.9974
概率分布: [0.002623174339532852, 0.9973767995834351]
--------------------------------------------------
文本: 收到 货 之后 光

In [ ]:
###以下加载不冻结参数的模型  


In [11]:
from transformers import BertTokenizer
# 从本地路径加载tokenizer
tokenizernofre = BertTokenizer.from_pretrained(tokenizernofre_dir)

# 使用方式同上
encoded_input_nofre = tokenizernofre("Your text here", return_tensors='pt')
encoded_input_nofre

{'input_ids': tensor([[  101,   100, 10539, 10815,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [12]:
import torch
from transformers import BertForSequenceClassification, BertConfig

# 加载模型 和配置
# 
config_nofre = BertConfig.from_pretrained(bert_model_nofre_dir)
model_nofre = BertForSequenceClassification.from_pretrained(
    bert_model_nofre_dir,
    config=config_nofre,
    use_safetensors=True
)

model_nofre.eval()  

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:

# 使用示例
label_map = {0: "负面", 1: "正面"}  # 根据实际情况修改
cmt_test=['商家发出的根本就不是一款，跟链接里的不一样，买的支架款也没有支架，真服了，发货巨慢，售后开个发票也磨磨叽叽到现在也没有拿到',     
          '保温效果一般，两个小时不到就凉了',
          '包装有脏污，盒子封口贴全是灰尘，送人用的，没办法只能把封口贴撕下来'
          ,'包裹运输中破损，持续20多天都没有退款',
          '纯纯塑料杯子，这杂能用，一股塑料的味道，劣质塑料',
          '有味道，不适合给孩子用，也不适合放热水，就是好看而已，真的一分钱一分货',
          '收到 货 之后 光顾 查询 序列号 去 没 仔细检查 外观 激活 之后 贴膜 才 发现 背 logo 上面 一点 刮伤 很 郁闷 很想 知道 刮痕',
          '风扇 噪音 大，没 想象 的 那么好',
          '这个保温杯真的超实用，670ml大容量满足全天饮水需求，316不锈钢材质安全无异味，保温效果长达12小时以上，粉色外观清新时尚。',
          '设计很漂亮，尺寸大小合适，包装仔细完整，手感很不错',
          '安装简单，价格实惠，宝贝确实不错，风力蛮大',
          '颜值高，白色很耐看，最喜欢的是米家的app远程控制，虽然需要有小爱音箱才能连接米家app,但是太好用了'
          ,'风量大声音小，可以调节风量，风速比较均匀，吹起来很舒服很自然',
         '期盼多久的镜子，太失望了',
          '包装有问题，把我手划破了',
          '除了好看一无是处',
          '太小了，装不了什么',]

model_nofre.to(device)
predictions_nofre = predict_with_model(model_nofre, cmt_test , tokenizernofre, device, label_map)

# 打印详细预测结果
for pred in predictions_nofre:
    print(f"文本: {pred['text']}")
    print(f"预测类别: {pred['predicted_label']} (索引:{pred['predicted_class_id']})")
    print(f"置信度: {pred['confidence']:.4f}")
    print(f"概率分布: {pred['probabilities']}")
    print("-" * 50)

文本: 商家发出的根本就不是一款，跟链接里的不一样，买的支架款也没有支架，真服了，发货巨慢，售后开个发票也磨磨叽叽到现在也没有拿到
预测类别: 负面 (索引:0)
置信度: 0.9994
概率分布: [0.9994184970855713, 0.0005814721807837486]
--------------------------------------------------
文本: 保温效果一般，两个小时不到就凉了
预测类别: 负面 (索引:0)
置信度: 0.9985
概率分布: [0.998454213142395, 0.0015457230620086193]
--------------------------------------------------
文本: 包装有脏污，盒子封口贴全是灰尘，送人用的，没办法只能把封口贴撕下来
预测类别: 负面 (索引:0)
置信度: 0.9948
概率分布: [0.994773805141449, 0.005226248409599066]
--------------------------------------------------
文本: 包裹运输中破损，持续20多天都没有退款
预测类别: 负面 (索引:0)
置信度: 0.9962
概率分布: [0.996221661567688, 0.0037782927975058556]
--------------------------------------------------
文本: 纯纯塑料杯子，这杂能用，一股塑料的味道，劣质塑料
预测类别: 负面 (索引:0)
置信度: 0.9987
概率分布: [0.9986698627471924, 0.0013301356229931116]
--------------------------------------------------
文本: 有味道，不适合给孩子用，也不适合放热水，就是好看而已，真的一分钱一分货
预测类别: 正面 (索引:1)
置信度: 0.9974
概率分布: [0.002623174339532852, 0.9973767995834351]
--------------------------------------------------
文本: 收到 货 之后 光